<a href="https://colab.research.google.com/github/KuJoJoTaRo2019/HaveFun/blob/master/Project3_short_semester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project3 is Social Network Analysis.**

I haven't understand this stock of theory. Sorry.

I will take some time to review.

---

In [ ]:
pip install -U python-igraph

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install py2neo

  Created wheel for neotime: filename=neotime-1.7.4-py3-none-any.whl size=20546 sha256=78a11735f545b06063c9610ee2a3502b3b07bde533b42918a0ac6ee933ca3603
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\1b\53\64\de72baae858a94a3ed2c6b1117edf988b2b8d57b798097e5b3
Successfully built neotime
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.3
    Uninstalling prompt-toolkit-3.0.3:
      Successfully uninstalled prompt-toolkit-3.0.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
ERROR: pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
ERROR: astroid 2.3.3 requires typed-ast<1.5,>=1.4.0; implementation_name == "cpython" and python_version < "3.8", which is not installed.


In [ ]:
from igraph import *
g = Graph([(0,1),(0,2),(2,3),(3,4),(4,2),(2,5),(5,0),(6,3),(5,6)]) # 生成图
print(type(g)) # 图的类型
summary(g) # 图的节点数和边数
print(g.degree()) # 各节点的度数

<class 'igraph.Graph'>
IGRAPH U--- 7 9 -- 
[3, 1, 4, 3, 2, 3, 2]


In [ ]:
from py2neo import Graph
#连接neo4j数据库，输入地址、用户名、密码
graph = Graph("http://localhost:7474", username="neo4j", password ='wangdejumen')

In [ ]:
print(graph)

Graph('http://neo4j@localhost:7474', name='neo4j')


In [ ]:
import csv
from igraph import Graph as IGraph

def read_data(filename):
    edges=[]
    firstline=True
    with open(filename,'r',encoding='utf-8') as f:
        for row in csv.reader(f.read().splitlines()):
            if firstline == True:
                firstline = False
                continue
            u,v,weight=[i for i in row]
            edges.append((u,v,int(weight)))
    return edges

def get_igraph(edges):
    g =IGraph.TupleList(edges=edges, directed=False, vertex_name_attr='name', edge_attrs=None, weights=True)
    return g

def get_graphData(graph):
    names = graph.vs['name']
    weights = graph.es['weight']
    print('人物名字：',names[:10])
    print('关系权重：',weights[:10])
    return names,weights

#度中心性
def Degree_Centrality(g):
    p=list(zip(g.vs['name'],g.vs.degree()))
    p_sort=sorted(p,key=lambda p:p[1],reverse=True)
    return p_sort

#加权度中心性
def Weight_Degree_Centrality(g,edges):
    WDC=[]
    for p in g.vs:
        neighbors = [x['name'] for x in p.neighbors()]
        weightdDegree = sum([w for a,b,w in edges for c in neighbors if (a==p['name'] and b==c) or (b==p['name'] and a==c)])
        WDC.append([p['name'],weightdDegree])
    WDC_sort = sorted(WDC,key=lambda p:p[1],reverse=True)
    return WDC_sort

#邻居平均度
def Neighbor_Average_Degree(g):
    p=list(zip(g.vs['name'],g.knn()[0]))
    p_sort=sorted(p,key=lambda p:p[1],reverse=True)
    return p_sort

#介数中心性
def Betweenness_Centrality(g):
    p=list(zip(g.vs['name'],g.betweenness()))
    p_sort=sorted(p,key=lambda p:p[1],reverse=True)
    return p_sort

#紧度中心性
def Closeness_centrality(g):
    p=list(zip(g.vs['name'],g.closeness()))
    p_sort=sorted(p,key=lambda p:p[1],reverse=True)
    return p_sort

#PageRank算法
def PageRank(g):
    p=list(zip(g.vs['name'],g.pagerank()))
    p_sort=sorted(p,key=lambda p:p[1],reverse=True)
    return p_sort

def Community_Detection(g):
    clusters = IGraph.community_walktrap(g,weights='weight').as_clustering()
    n=len(clusters.sizes())
    for i in range(n):
        print('Community',i,':',[g.vs['name'][j] for j in clusters[i]])

if __name__=='__main__':
    edges=read_data('./stormofswords.csv')
    g=get_igraph(edges)
    names,weights = get_graphData(g)
    print('------------------------------------------------------')
    print('角色数：',g.vcount())
    print('网络直径：',g.diameter())
    print('直径上的角色：',[names[x] for x in g.get_diameter()])
    print('------------------------------------------------------')
    print('其中一条最短路径：',[names[x] for x in g.get_shortest_paths('Catelyn','Drogo')[0]])
    print('\r 所有最短路径：')
    paths=g.get_all_shortest_paths('Catelyn','Drogo')
    for p in paths:
        print([names[x] for x in p])
    print('------------------------------------------------------')
    #中心线度量
    print('度中心性：',Degree_Centrality(g)[:10])
    print('加权度中心性：',Weight_Degree_Centrality(g,edges)[:10])
    print('邻居平均度：',Neighbor_Average_Degree(g)[:10])
    print('介数中心性：',Betweenness_Centrality(g)[:10])
    print('紧度中心性：',Closeness_centrality(g)[:10])
    print('PageRank算法：',PageRank(g)[:10])
    print('------------------------------------------------------')
    print('社区划分结果：')
    Community_Detection(g)

人物名字： ['Aemon', 'Grenn', 'Samwell', 'Aerys', 'Jaime', 'Robert', 'Tyrion', 'Tywin', 'Alliser', 'Mance']
关系权重： [5, 31, 18, 6, 5, 8, 5, 5, 11, 23]
------------------------------------------------------
角色数： 107
网络直径： 6
直径上的角色： ['Amory', 'Oberyn', 'Tyrion', 'Jaime', 'Barristan', 'Belwas', 'Illyrio']
------------------------------------------------------
其中一条最短路径： ['Catelyn', 'Jaime', 'Robert', 'Daenerys', 'Drogo']
 所有最短路径：
['Catelyn', 'Jaime', 'Barristan', 'Jorah', 'Drogo']
['Catelyn', 'Eddard', 'Robert', 'Daenerys', 'Drogo']
['Catelyn', 'Stannis', 'Robert', 'Daenerys', 'Drogo']
['Catelyn', 'Sansa', 'Robert', 'Daenerys', 'Drogo']
['Catelyn', 'Cersei', 'Robert', 'Daenerys', 'Drogo']
['Catelyn', 'Tyrion', 'Robert', 'Daenerys', 'Drogo']
['Catelyn', 'Jaime', 'Robert', 'Daenerys', 'Drogo']
['Catelyn', 'Jaime', 'Barristan', 'Daenerys', 'Drogo']
['Catelyn', 'Tyrion', 'Viserys', 'Daenerys', 'Drogo']
------------------------------------------------------
度中心性： [('Tyrion', 36), ('Jon', 26), ('Sansa'